In [1]:
%matplotlib notebook

import sys
from typing import List, Tuple, Optional
from configparser import ConfigParser
import random
import time

sys.path.insert(0, "../../../global_route_planner/")

import numpy as np
import carla
from agents.navigation.local_planner import LocalPlanner, _compute_connection
from shapely.geometry import LineString, Point
from matplotlib import pyplot as plt
from IPython import display

from shapely.geometry import LineString, Point

from global_planner import get_client, spawn_vehicle, draw_waypoints, get_global_planner
from cartesian_to_frenet import get_frenet_from_cartesian, get_path_linestring

sys.path.insert(0, "../../../carla_utils/utils/")


from actors import Actor, Vehicle

In [2]:
def filter_waypoints(waypoints, road_id, lane_id = None):
    
    filtered_waypoints = []
    for wp in waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            filtered_waypoints.append(wp) 
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            filtered_waypoints.append(wp) 

    return filtered_waypoints


def spawn_vehicle(
    world: carla.libcarla.World,
    vehicle_type: str = "model3",
    spawn_point: Optional[carla.libcarla.Transform] = None,
) -> carla.libcarla.Vehicle:
    """
    Spawns a vehicle at a given spawn point. Default car model is 'model3'.
    If no spawn point is provided, randomly selects the spawn point from the set of pre-assigned spawn points in the map.
    """

    if spawn_point is None:
        spawn_point = random.choice(world.get_map().get_spawn_points())
    vehicle_blueprint = world.get_blueprint_library().filter(vehicle_type)[0]
    vehicle = world.spawn_actor(vehicle_blueprint, spawn_point)
    return vehicle

In [3]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

Connection to CARLA server established!


In [278]:
for actors in world.get_actors().filter('vehicle*'):
    actors.destroy()

In [279]:
all_waypoints = world.get_map().generate_waypoints(1)

filtered_waypoints = filter_waypoints(all_waypoints, 37, -2)

draw_waypoints(world, filtered_waypoints[300:-300], 2)

In [280]:
wps = np.random.choice(filtered_waypoints[300:-300], 6)
draw_waypoints(world, wps, 2)

In [281]:
actors = []
for wp in wps:
    
    x = wp.transform.location.x
    y = wp.transform.location.y
    z = wp.transform.location.z + 5
    yaw = wp.transform.rotation.yaw
    transform = carla.Transform(carla.Location(x=x,y=y,z=z), carla.Rotation(yaw=yaw))

    actors.append(spawn_vehicle(world, 'model3', transform))
    
import time
time.sleep(1)
    
actors = [Vehicle(world, actor.id) for actor in actors]

In [282]:
[actors[i].location for i in range(len(actors))]

[{'x': -243.78048706054688, 'y': 68.41490936279297, 'z': 9.892717361450195},
 {'x': -244.10540771484375, 'y': 91.41178131103516, 'z': 9.836559295654297},
 {'x': -243.7767791748047, 'y': -60.42034912109375, 'z': 9.896491050720215},
 {'x': -243.24850463867188, 'y': -24.424224853515625, 'z': 9.895075798034668},
 {'x': -243.34292602539062, 'y': 37.4180908203125, 'z': 9.975760459899902},
 {'x': -244.1912384033203, 'y': -88.41825866699219, 'z': 10.251269340515137}]

In [283]:
idx = -1
draw_waypoints(world, [wps[idx]], 10)


print(wps[idx].transform.location.x, wps[idx].transform.location.y)

ego_vehicle = actors[idx]
ego_lane_linestring = get_path_linestring(filtered_waypoints)
ego_vehicle_location_point = Point(ego_vehicle.location['x'], ego_vehicle.location['y'])
ego_vehicle_heading = ego_vehicle.location_global.theta
ego_vehicle_frenet_coordinates = get_frenet_from_cartesian(ego_lane_linestring, ego_vehicle_location_point, ego_vehicle_heading)
ego_vehicle_frenet_coordinates

-244.19125366210938 -88.41825866699219


(322.30105885401946,
 -1.5257131360471183e-05,
 0.00014027504447255978,
 <shapely.geometry.point.Point at 0x7f05e29aa748>)

(238.58229692105292,
 -9.69647949388488e-06,
 0.00030378135781461424,
 <shapely.geometry.point.Point at 0x7f05e29f6da0>)